

*   **Creador:** Juan Diego Cuadros Rodriguez



### Punto 1

In [ ]:
!pip install --quiet yfinance

In [ ]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [ ]:
import yfinance as yf

In [ ]:
# Importamos los activos que vamos a análizar, con la fecha de iniciio de la 
# fecha de inicio de la serie y la final
assets = ['GS','BIIB']
start = "2021-01-01"
end = "2022-12-31"

In [ ]:
yf.pdr_override() # este proceso se hace con el fin de que DataReader tome la estructura de yfinance
#Importamos los datos diarios
data_daily=web.DataReader(assets,start,end)['Adj Close'] 
# Importamos los datos mensuales
data_mo = web.DataReader(assets,start= start, end=end, interval='1Mo')['Adj Close']
data_mo

In [ ]:
# Calculamos los logaritmos diarios, mensuales y anuales
ret_daily = np.log(data_daily/data_daily.shift(1)).dropna()
ret_mon = np.log(data_mo/data_mo.shift(1)).dropna()
re_anu = ret_mon * 12
ret_anual = re_anu.mean()

In [ ]:
# Tabla resumen
tabla = pd.DataFrame([ret_daily.mean(),ret_mon.mean(),ret_anual],index={'Retorno diario','Retorno mensual','Retorno anual'})
tabla

In [ ]:
# Retorno medio anualizado
ret_anual

In [ ]:
# Volatilidad Anual
vol = ret_mon.std() * np.sqrt(12)
vol

In [ ]:
# tabla de datos en estadistica descriptiva
# Retornos diarios esperado
ret_med_daily = ret_daily.mean()
vol_daily = ret_daily.std()
median_daily = ret_daily.median()
kurt_ret_daily = ret_daily.kurtosis()
skw_ret_daily = ret_daily.skew()

data2=[median_daily,ret_med_daily,vol_daily,skw_ret_daily,kurt_ret_daily]
index={"E(r)","Vol(r)","Med(r)","Kurt","Skew"}

In [ ]:
total_summary = pd.DataFrame(data2,index)
total_summary

In [ ]:
from scipy.stats import norm
x = np.linspace(-0.1,0.1,num=1000)
plt.hist(ret_daily['BIIB'],bins=20,density=True,histtype='step')
plt.ylabel('Retornos')
plt.xlabel('Probabilidades')
plt.plot(x,norm.pdf(x,scale=ret_daily['BIIB'].std()),'k')
plt.axvline(ret_daily['BIIB'].mean(),color='r');
plt.show()



*  La distribución los retornos de Biogin tiene un sesgo positivo, debido a que presenta un outlier mayor a 0.3
*  La distribución de los datos de Biogen tienen un exceso de curtosis, por lo tanto la distribución es leptocurtica
*  De esta manera los datos tienden a su media y, asimismo, tiene mayor probabilidad de caer en los extremos



In [ ]:
x = np.linspace(-0.1,0.1,num=1000)
plt.hist(ret_daily['GS'],bins=20,density=True,histtype='step')
plt.ylabel('Retornos')
plt.xlabel('Probabilidades')
plt.plot(x,norm.pdf(x,scale=ret_daily['GS'].std()),'k')
plt.axvline(ret_daily['GS'].mean(),color='r');
plt.show()



*  La distribución no tiene sesgos significativos en la parte positiva o negativa
*  La distribución no presenta outliers
*  Asimismo, la distribución presenta una kurtosis mayor a cero, por lo tanto, sigue una distribución leptokurtica



In [ ]:
# Calculamos la matriz de correlación de Pearson
ret_daily.corr()

Segun la correlación de Person, los dos activos tienen correlación positiva; sin embargo, debido a ciertos outliers, el coeficiente es alterado

In [ ]:
# Calculamos la matriz de correlación de Kendall
ret_daily.corr(method='kendall')

Segun la correlación de Rango de Kendall la concordancia que tienen los
activos es menor, por lo tanto, sus movimientos que siguen entre sí
Diferen más entre ellos

In [ ]:
# Calculamos la correlación de Spearman
ret_daily.corr(method='spearman')

A partir de este resultado, se puede concluir que los datos con base es la diferencia de sus retornos tienen mayor relación, por lo tanto, tienen correlación positiva, más alta que los demás

### Punto 2

In [ ]:
ticker = ['GS','JPM','TSLA','AAPL','MSFT','C','BIIB','JNJ','F','NVDA']
start = '2017-12-31'
end = '2022-12-31'

In [ ]:
shares = yf.download(ticker,start=start,end=end,interval='1Mo')['Adj Close']
returns = np.log(shares/shares.shift(1)).dropna()

In [ ]:
e_r = returns.mean().sort_values(ascending=False)
e_v = returns.std().sort_values(ascending=False)
e_r
e_v

In [ ]:
ind={'Volatility','return'}

In [ ]:
df1=pd.DataFrame([e_r,e_v],index=ind)

In [ ]:
# Segun lo 
df1

### Punto 3

In [ ]:
# Obtenemos los datos pertinentes
index='^RUT'
start = '2011-12-31'
end = '2022-12-31'
yf.pdr_override()
ind = web.DataReader(index,start,end,interval='1Mo')
ind

**Desviación Estandar**

In [ ]:
# Obtenemos los retornos mensualess y luego de ello aplicamos el metodo para encontrar la 
# desviación estandar, mensual
returns1 = np.log(ind['Adj Close']/ind['Adj Close'].shift(1)).dropna()
vol_cc_m = returns1.std()
# Luego de ello, anualizamos la desviación estandar con la raiz del numero de meses
vol_cc_a = vol_cc_m*np.sqrt(12)

In [ ]:
# la desviación no es constante en el tiempo
sig_r = returns1.rolling(window=12).std()
sig_r.plot()

In [ ]:
# Desviación estandar mensual
vol_cc_m

La volatilidad media mensual es de 0.05469066

```
# Tiene formato de código
```



In [ ]:
# Desviación estandar anualizada
vol_cc_a

La volatilidad anual del indice es 0.189454, no obstante, no refleja la variabilidad real de los datos, debido a que no es constate

**Volatilidad de Parkinson**

In [ ]:
# Función para realizar la volatilidad de Parkinson 
def vol_park(stock):
  lnhl = np.log(stock['High']/stock['Low']).dropna()
  sigmapark_m = (((1/(4*np.log(2)))*lnhl**2)**0.5).mean()
  return(sigmapark_m)

In [ ]:
# Volatilidad de Parkinson mensual
vol_park(ind)

In [ ]:
# Volatilidad de Parkinson anualizada
vol_park(ind)*np.sqrt(12)

La volatilidad de Parkinson es explicada por la diferencia entre el precio de más alto y más bajo, por lo tanto, el resultado de 0.177025 indica que la volatilidad presentada entrer los precios es menor

**Volatilidad de Garman-Klass**

In [ ]:
# Función para realizar la volatilidad de Garman-Klass mensual
def vol_gk(stock):
  lnlh=np.log(stock['High']/stock['Low']).dropna()
  lnco = np.log(stock['Close']/stock['Open']).dropna()
  sigmagk_m = ((0.5*lnlh**2-(2*np.log(2)-1)*lnco**2)**0.5).mean()
  return(sigmagk_m)

In [ ]:
# Volatilidad de Garman-Klass mensual
vol_gk(ind)

In [ ]:
# Volatilidad de Garman-Klass anual
vol_gk(ind)*np.sqrt(12)

Analizando los cambios precios entre cierre-apertura y alto-bajo a lo largo de la serie, se puede afirmar que la volatilidad ha sido alta y afectada por la esstacionalidad de estos retornos

**EWMA**

In [ ]:
# A traves de esta función se automatizó el proceso para obtener la volatilidad EWMA mensual
def ewma_vol(returns_f,l):
  k = len(returns_f)
  td = np.arange(k-1,-1,-1)
  alpha =l**td
  var_ewma = ((returns_f**2*alpha).sum())*(1-l)
  vol_ewma = np.sqrt(var_ewma)
  return(vol_ewma)

In [ ]:
# Se define que lamda se va a utilizar y con ello, se obtiene la volatilidad EWMA mensual
lamda = 0.94
ewma_vol(returns1,lamda)

In [ ]:
# Obtenemos la Volatilidad EWMA anualizada multiplicada por la raiz de los 12 meses
ewma_vol(returns1,lamda)*np.sqrt(12)

Anteriormente se evidencio que la volatilidad del indice era mayor en el ultimo tiempo. A traves de la volatilidad EWMA anualizada, asignando un mayor peso a los ultimos 30 días, se puede concluir que la volatilidad del indice ha sido mayor que en los demás metodos